# SQL Server 2019 Data Virtualization - Using Polybase to query Azure SQL Database
This notebook contains an example of how to use external tables to query data in Azure SQL Database without moving data. You may need to change identity, secret, connection, database, schema, and remote table names to work with your Azure SQL Database.

This notebook also assumes you are using SQL Server 2019 Release Candidate or later and that the Polybase feature has been installed and enabled.

This notebook uses the sample WideWorldImporters sample database but can be used with any user database.

## Step 0: Create a database in Azure SQL, table, and add data

Create a database in Azure SQL called **wwiazure**. Execute the following T-SQL to create a table and insert a row in the database

```sql
DROP TABLE IF EXISTS [ModernStockItems]
GO
CREATE TABLE [ModernStockItems](
	[StockItemID] [int] NOT NULL,
	[StockItemName] [nvarchar](100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[SupplierID] [int] NOT NULL,
	[ColorID] [int] NULL,
	[UnitPackageID] [int] NOT NULL,
	[OuterPackageID] [int] NOT NULL,
	[Brand] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[Size] [nvarchar](20) COLLATE Latin1_General_100_CI_AS NULL,
	[LeadTimeDays] [int] NOT NULL,
	[QuantityPerOuter] [int] NOT NULL,
	[IsChillerStock] [bit] NOT NULL,
	[Barcode] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[TaxRate] [decimal](18, 3) NOT NULL,
	[UnitPrice] [decimal](18, 2) NOT NULL,
	[RecommendedRetailPrice] [decimal](18, 2) NULL,
	[TypicalWeightPerUnit] [decimal](18, 3) NOT NULL,
	[LastEditedBy] [int] NOT NULL,
CONSTRAINT [PK_Warehouse_StockItems] PRIMARY KEY CLUSTERED 
(
	[StockItemID] ASC
)
)
GO
-- Now insert some data. We don't coordinate with unique keys in WWI on SQL Server
-- so pick numbers way larger than exist in the current StockItems in WWI which is only 227
INSERT INTO ModernStockItems VALUES
(100000,
'Dallas Cowboys Jersey',
5,
4, -- Blue
4, -- Box
4, -- Bob
'Under Armour',
'L',
30,
1,
0,
'123456789',
2.0,
50,
75,
2.0,
1
)
GO```


## Step 1: Create a master key
Create a master key to encrypt the database credential

In [2]:
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

## Step 2: Create a database credential.
The database credential contains the IDENTITY (login) and SECRET (password) of the remote Azure SQL Database server or Managed Instance. Change this to the login and password for your server.

In [3]:
CREATE DATABASE SCOPED CREDENTIAL AzureSQLDatabaseCredentials   
WITH IDENTITY = '<login>', SECRET = '<password>'
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 3: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

The LOCATION syntax is <datasourcetype>:<connection string>.

datasourcetype can be sqlserver, oracle, teradata, mongodb, or odbc (Windows only)
The connection string depends on the datasourcetype

For this example, put in the name of the Azure SQL Server database server or Managed instance

In [4]:
CREATE EXTERNAL DATA SOURCE AzureSQLDatabase
WITH ( 
LOCATION = 'sqlserver://<server name>.database.windows.net',
PUSHDOWN = ON,
CREDENTIAL = AzureSQLDatabaseCredentials
)
GO

Commands completed successfully.

Total execution time: 00:00:00.942

## Step 4: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [5]:
CREATE SCHEMA azuresqldb
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 5: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. The name of the table for the external table can be your choice. But the columns must be specified with the same name as they are defined in the remote table. Furthermore, local data types must be compatible with the remote table.

The WITH clause specifies a LOCATION. This LOCATION is different than the EXTERNAL DATA SOURCE. This LOCATION indicates the [database].[schema].[table] of the remote table. The DATA_SOURCE clauses is the name of the EXTERNAL DATA SOURCE you created earlier.

In [6]:
CREATE EXTERNAL TABLE azuresqldb.ModernStockItems
(
	[StockItemID] [int] NOT NULL,
	[StockItemName] [nvarchar](100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[SupplierID] [int] NOT NULL,
	[ColorID] [int] NULL,
	[UnitPackageID] [int] NOT NULL,
	[OuterPackageID] [int] NOT NULL,
	[Brand] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[Size] [nvarchar](20) COLLATE Latin1_General_100_CI_AS NULL,
	[LeadTimeDays] [int] NOT NULL,
	[QuantityPerOuter] [int] NOT NULL,
	[IsChillerStock] [bit] NOT NULL,
	[Barcode] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[TaxRate] [decimal](18, 3) NOT NULL,
	[UnitPrice] [decimal](18, 2) NOT NULL,
	[RecommendedRetailPrice] [decimal](18, 2) NULL,
	[TypicalWeightPerUnit] [decimal](18, 3) NOT NULL,
	[LastEditedBy] [int] NOT NULL
)
 WITH (
 LOCATION='wwiazure.dbo.ModernStockItems',
 DATA_SOURCE=AzureSQLDatabase
)
GO

Commands completed successfully.

Total execution time: 00:00:01.217

## Step 6: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [7]:
CREATE STATISTICS ModernStockItemsStats ON azuresqldb.ModernStockItems ([StockItemID]) WITH FULLSCAN
GO

Commands completed successfully.

Total execution time: 00:00:01.597

## Step 7: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [8]:
SELECT * FROM azuresqldb.ModernStockItems
GO

(1 row affected)

Total execution time: 00:00:01.369

StockItemID,StockItemName,SupplierID,ColorID,UnitPackageID,OuterPackageID,Brand,Size,LeadTimeDays,QuantityPerOuter,IsChillerStock,Barcode,TaxRate,UnitPrice,RecommendedRetailPrice,TypicalWeightPerUnit,LastEditedBy
100000,Dallas Cowboys Jersey,5,4,4,4,Under Armour,L,30,1,0,123456789,2.000,50.00,75.00,2.000,1


## Step 8: Query the remote table with a WHERE clause
Even though the table may be small SQL Server will "push" the WHERE clause filter to the remote table

In [9]:
SELECT * FROM azuresqldb.ModernStockItems WHERE StockItemID = 100000
GO

(1 row affected)

Total execution time: 00:00:01.612

StockItemID,StockItemName,SupplierID,ColorID,UnitPackageID,OuterPackageID,Brand,Size,LeadTimeDays,QuantityPerOuter,IsChillerStock,Barcode,TaxRate,UnitPrice,RecommendedRetailPrice,TypicalWeightPerUnit,LastEditedBy
100000,Dallas Cowboys Jersey,5,4,4,4,Under Armour,L,30,1,0,123456789,2.000,50.00,75.00,2.000,1


## Step 9: Join with local SQL Server tables
Use a UNION to find all stockitems for a specific supplier both locally and in the Azure table

In [10]:
SELECT msi.StockItemName, msi.Brand, c.ColorName
FROM azuresqldb.ModernStockItems msi
JOIN [Purchasing].[Suppliers] s
ON msi.SupplierID = s.SupplierID
and s.SupplierName = 'Graphic Design Institute'
JOIN [Warehouse].[Colors] c
ON msi.ColorID = c.ColorID
UNION
SELECT si.StockItemName, si.Brand, c.ColorName
FROM [Warehouse].[StockItems] si
JOIN [Purchasing].[Suppliers] s
ON si.SupplierID = s.SupplierID
and s.SupplierName = 'Graphic Design Institute'
JOIN [Warehouse].[Colors] c
ON si.ColorID = c.ColorID
GO

(43 rows affected)

Total execution time: 00:00:00.306

StockItemName,Brand,ColorName
Dallas Cowboys Jersey,Under Armour,Blue
DBA joke mug - daaaaaa-ta (Black),NULL,Black
DBA joke mug - daaaaaa-ta (White),NULL,White
DBA joke mug - I will get you in order (Black),NULL,Black
DBA joke mug - I will get you in order (White),NULL,White
DBA joke mug - it depends (Black),NULL,Black
DBA joke mug - it depends (White),NULL,White
DBA joke mug - mind if I join you? (Black),NULL,Black
DBA joke mug - mind if I join you? (White),NULL,White
DBA joke mug - SELECT caffeine FROM mug (Black),NULL,Black
